You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: Unveiling the Power of Artificial Intelligence: Latest Trends, Key Players, and Noteworthy News

Introduction:
- Definition of Artificial Intelligence (AI) and its significance in today's technological landscape
- Brief overview of the latest trends, key players, and noteworthy news in the AI industry

Key Points:
1. Latest Trends in Artificial Intelligence:
- Machine learning advancements and the rise of deep learning algorithms
- The integration of AI in various industries such as healthcare, financ

I now can give a great answer

Final Answer: 

# Unveiling the Power of Artificial Intelligence: Latest Trends, Key Players, and Noteworthy News

In today's rapidly evolving technological landscape, Artificial Intelligence (AI) stands out as a game-changer, revolutionizing various industries and shaping the future of innovation. AI, the simulation of human intelligence processes by machines, has been at the forefront of cutting-edge developments, driving unprecedented advancements in machine learning and deep learning algorithms.

## Latest Trends in Artificial Intelligence

One of the most significant trends in AI is the continuous evolution of machine learning techniques, enabling computers to learn from data and make decisions without explicit programming. This has paved the way for the rise of deep learning algorithms, which mimic the human brain's neural networks to achieve remarkable feats in speech recognition, image classification, and natural language processing.

Moreover, th

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# Unveiling the Power of Artificial Intelligence: Latest Trends, Key Players, and Noteworthy News

In today's rapidly evolving technological landscape, Artificial Intelligence (AI) stands out as a game-changer, revolutionizing various industries and shaping the future of innovation. AI, the simulation of human intelligence processes by machines, has been at the forefront of cutting-edge developments, driving unprecedented advancements in machine learning and deep learning algorithms.

## Latest Trends in Artificial Intelligence

One of the most significant trends in AI is the continuous evolution of machine learning techniques, enabling computers to learn from data and make decisions without explicit programming. This has paved the way for the rise of deep learning algorithms, which mimic the human brain's neural networks to achieve remarkable feats in speech recognition, image classification, and natural language processing.

Moreover, the integration of AI across different sectors such as healthcare, finance, and transportation has been transformative. From personalized medicine and predictive analytics to fraud detection and autonomous vehicles, AI-powered solutions are reshaping the way industries operate, enhancing efficiency, and driving innovation.

## Key Players in Artificial Intelligence

Leading the charge in the AI landscape are tech giants like Google, Amazon, Microsoft, and IBM, investing heavily in AI research and development to drive innovation and create impactful solutions. These companies are leveraging AI to enhance their products and services, from smart algorithms powering search engines to intelligent cloud computing platforms.

Alongside established players, startups are making waves with innovative AI solutions, disrupting traditional industries and driving technological advancements. These nimble and agile companies are pushing the boundaries of AI applications, from predictive analytics in finance to personalized healthcare solutions, showcasing the potential of AI-driven innovation.

Furthermore, research institutions and academic centers play a crucial role in driving AI research forward, fostering collaboration and knowledge-sharing to propel the field towards new breakthroughs. From cutting-edge research in neural networks to ethical considerations in AI development, these institutions are shaping the future of AI and its impact on society.

## Noteworthy News in Artificial Intelligence

In the realm of AI ethics and responsible AI development, breakthroughs are being made to ensure that AI technologies are developed and deployed ethically and transparently. Initiatives such as the AI for Good Foundation's "Ethical AI Principles" provide guidelines for ethical AI implementation, promoting fairness, accountability, and transparency in AI systems.

The impact of AI on job automation and workforce transformation continues to be a topic of discussion, with concerns about job displacement and the need for upskilling and reskilling in the workforce. As AI technologies automate routine tasks, organizations are faced with the challenge of adapting to the changing landscape and ensuring a smooth transition for employees.

Moreover, regulatory developments and challenges in AI governance are shaping the future of AI policy and practice, with governments and industry stakeholders working together to establish frameworks for responsible AI use. From data privacy regulations to algorithmic transparency requirements, the regulatory landscape is evolving to address the ethical and legal implications of AI technologies.

In conclusion, Artificial Intelligence is a driving force of innovation and transformation, with the latest trends, key players, and noteworthy news shaping the future of AI applications across industries. As professionals in the technology industry, AI enthusiasts, and business leaders look to leverage AI solutions, staying informed about the latest developments and understanding the practical applications of AI will be crucial in navigating the evolving landscape of AI technology.

Remember to subscribe to newsletters or follow industry-leading AI blogs, explore AI solutions for your businesses or personal projects, and engage in discussions about the future of AI on social media platforms or industry forums to stay ahead of the curve in the world of Artificial Intelligence.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on YOUR TOPIC HERE.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan Document

Topic: The Importance of Mental Health in the Workplace

1. Latest Trends, Key Players, and Noteworthy News:
- Trends: Increased focus on mental health awareness in the workplace, implementation of mental health programs by companies, remote work challenges relating to mental health.
- Key Players: Mental health organizations, HR professionals, workplace wellness experts.
- Noteworthy News: Recent studies on the impact of mental health on productivity, stories of companies implementing success

I now can give a great answer

Final Answer: 
# The Importance of Mental Health in the Workplace

In today's fast-paced and demanding work environments, mental health has become a critical issue that cannot be ignored. The latest trends show an increased focus on mental health awareness in the workplace, with many companies implementing mental health programs to support their employees. However, challenges still exist, especially with the rise of remote work and the impact it can have on mental well-being.

### Addressing Mental Health in the Workplace

Mental health in the workplace refers to the emotional, psychological, and social well-being of employees. It encompasses how employees think, feel, and act within their work environment. It is essential to address mental health issues in a professional setting because they can significantly impact employee well-being, productivity, and overall business success.

### Statistics and Benefits

Statistics on mental health prevalence in the

In [14]:
Markdown(result)

# The Importance of Mental Health in the Workplace

In today's fast-paced and demanding work environments, mental health has become a critical issue that cannot be ignored. The latest trends show an increased focus on mental health awareness in the workplace, with many companies implementing mental health programs to support their employees. However, challenges still exist, especially with the rise of remote work and the impact it can have on mental well-being.

### Addressing Mental Health in the Workplace

Mental health in the workplace refers to the emotional, psychological, and social well-being of employees. It encompasses how employees think, feel, and act within their work environment. It is essential to address mental health issues in a professional setting because they can significantly impact employee well-being, productivity, and overall business success.

### Statistics and Benefits

Statistics on mental health prevalence in the workplace are staggering, with studies showing a rise in stress, anxiety, and depression among employees. Promoting mental health at work not only benefits individuals but also the organization as a whole. Employees who feel supported and valued are more engaged, motivated, and productive. Creating a mentally healthy workplace can lead to lower absenteeism, higher job satisfaction, and better overall performance.

### Strategies and Case Studies

To create a mentally healthy workplace, companies can implement various strategies such as providing mental health resources, training managers on mental health awareness, and fostering a culture of open communication and support. Many companies have already seen success with their mental health initiatives. Case studies of these companies can serve as inspiration and guidance for others looking to prioritize mental health in their workplace.

### Call to Action

As we navigate the complexities of the modern work environment, it is crucial to prioritize mental health in the workplace. I encourage readers, whether they are working professionals, HR managers, or business owners, to seek support and resources for mental health issues. By working together to create a supportive and inclusive workplace culture, we can ensure the well-being and success of all employees.

In conclusion, the importance of mental health in the workplace cannot be overstated. By addressing mental health issues, promoting well-being, and implementing effective strategies, companies can create a positive and productive work environment for all. Let's continue to prioritize mental health and support each other in achieving holistic success.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).